# State space representation

The "standard" or most commonly used state space representation is 

\begin{align} \dot{x} &= Ax + Bu \\ y &= Cx + Du \end{align}

Take note that Seborg uses a slightly different version:

\begin{align} \dot{x} &= Ax + Bu + Ed\\ y &= Cx \end{align}

This second version can not represent pure gain systems as it effectively assumes $D=0$. It is also possible to stack $u$ and $d$ from the bottom form into one input vector, so the $E$ matrix really doesn't add much. As you may infer, I prefer the top version and it is also the version used by most libraries.

In [1]:
import numpy
import numpy.linalg

## Converting between state space and transfer function forms

### Scipy.signal

The `scipy.signal` library handles conversion between transfer function coefficients and state space matrices easily:

In [2]:
import scipy.signal

In [3]:
G = scipy.signal.lti(1, [1, 1])

In [4]:
G

TransferFunctionContinuous(
array([ 1.]),
array([ 1.,  1.]),
dt: None
)

This single object allows us to access the numerator and denominator

In [5]:
G.num, G.den

(array([ 1.]), array([ 1.,  1.]))

And also the state space matrices

In [6]:
G.A, G.B, G.C, G.D

(array([[-1.]]), array([[ 1.]]), array([[ 1.]]), array([[ 0.]]))

We can build another object using the state space matrices instead of the Laplace form

In [7]:
G2 = scipy.signal.lti(G.A, G.B, G.C, G.D)
G2

StateSpaceContinuous(
array([[-1.]]),
array([[ 1.]]),
array([[ 1.]]),
array([[ 0.]]),
dt: None
)

And we still have access to the numerator and denominator (there is a small warning about bad coefficients, but the answer is reliable).

In [8]:
G2.num, G2.den

/Users/alchemyst/anaconda3/lib/python3.6/site-packages/scipy/signal/filter_design.py:1452: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  "results may be meaningless", BadCoefficients)


(array([ 1.]), array([ 1.,  1.]))

Instead of building objects we can also use the functions in `scipy.signal.lti_conversion`:

In [9]:
scipy.signal.lti_conversion.tf2ss(1, [1, 1])

(array([[-1.]]), array([[ 1.]]), array([[ 1.]]), array([[ 0.]]))

In [10]:
scipy.signal.lti_conversion.ss2tf(-1, 1, 1, 0)

(array([[0, 1]]), array([ 1.,  1.]))

### Control library

The control library is supposed to handle this pretty easily as well:

**Note** There appears to be an error in the control library (which I have reported [here](https://github.com/python-control/python-control/issues/156)) when converting the examples I used in the `scipy.signal` section. My recommendation is to use that library if you see errors when using the control library.

In [11]:
import control

In [12]:
Gtf = control.tf([1, 1], [1, 2, 1, 1])
Gtf


       s + 1
-------------------
s^3 + 2 s^2 + s + 1

This object does not have .A, .B, .C and .D properties.

In [13]:
Gtf.A

AttributeError: 'TransferFunction' object has no attribute 'A'

We must instead convert the system using `ss` (short for state space) to gett a State Space representation:

In [14]:
Gss = control.ss(Gtf)
Gss

A = [[-2. -1. -1.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]]

B = [[ 1.]
 [ 0.]
 [ 0.]]

C = [[ 0.  1.  1.]]

D = [[ 0.]]

In [15]:
Gss.A

matrix([[-2., -1., -1.],
        [ 1.,  0.,  0.],
        [ 0.,  1.,  0.]])

## Symbolic conversion

It is easy to convert state space models to transfer functions since the Laplace transform is a linear operator:

$$ \dot{x} = Ax + Bu \quad \therefore \quad sX(s) = AX(s) + BU(s) \quad X(s) = (sI - A)^{-1}BU(s)$$
$$ y = Cx + Du \quad \therefore \quad Y(s) = CX(s) + DU(s) \quad Y(s) = \underbrace{(C(sI - A)^{-1}B + D)}_{G(s)}U(s)$$


In [16]:
import sympy

In [17]:
s = sympy.Symbol('s')

In [18]:
A, B, C, D = [sympy.Matrix(m) for m in [Gss.A, Gss.B, Gss.C, Gss.D]]

In [19]:
I = sympy.eye(3)

In [20]:
G = C*(s*I - A).inv()*B + D
sympy.nsimplify(G[0,0]).simplify()

(s + 1)/(s**3 + 2*s**2 + s + 1)

## Analysis

Notice that the roots of the characteristic function correspond with the eigenvalues of the A matrix. The numerator and denominator of control transfer functions are stored as lists of lists to accomodate MIMO systems.

In [21]:
Gtf.pole()

array([-1.75487767+0.j        , -0.12256117+0.74486177j,
       -0.12256117-0.74486177j])

In [22]:
numpy.roots(Gtf.den[0][0])

array([-1.75487767+0.j        , -0.12256117+0.74486177j,
       -0.12256117-0.74486177j])

In [23]:
numpy.linalg.eig(Gss.A)

(array([-1.75487767+0.j        , -0.12256117+0.74486177j,
        -0.12256117-0.74486177j]),
 matrix([[-0.83619532+0.j        , -0.39217250-0.13264918j,
          -0.39217250+0.13264918j],
         [ 0.47649778+0.j        , -0.08904282+0.54115503j,
          -0.08904282-0.54115503j],
         [-0.27152763+0.j        ,  0.72651740+0.j        ,  0.72651740-0.j        ]]))